In [2]:
import urllib.request, json
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToJson 
import pandas as pd

In [3]:
hdr ={
# Request headers
'Cache-Control': 'no-cache',
'api_key': 'c6c3671249fc4598aaaf0853495b851a',
}

In [4]:
#GTFS-RT

def get():
    def get_realtime_update():
        url = "https://api.wmata.com/gtfs/rail-gtfsrt-vehiclepositions.pb"

        req = urllib.request.Request(url, headers=hdr)
        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)

        # Parse the protobuf response
        feed = gtfs_realtime_pb2.FeedMessage()
        feed.ParseFromString(response.read())

        # Convert the FeedMessage protobuf object to JSON format
        feed_json = MessageToJson(feed)
        return feed_json

    def get_predictions():
        url = "https://api.wmata.com/StationPrediction.svc/json/GetPrediction/All"
        req = urllib.request.Request(url, headers=hdr)
        req.get_method = lambda: 'GET'
        response = urllib.request.urlopen(req)
        return response.read()
    
    return json.loads(get_realtime_update()), json.loads(get_predictions())

In [5]:
feed_rt, predictions = get()

In [6]:
def get_predictions():
    url = "https://api.wmata.com/StationPrediction.svc/json/GetPrediction/All"
    req = urllib.request.Request(url, headers=hdr)
    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    return response.read()

In [7]:
class Line:
    def __init__(self, name, code, color, trains, stations, trips):
        self.name = name
        self.code = code
        self.color = color
        self.trains = trains
        self.stations = stations  # A list of Station objects
        self.trips = trips
        #self.predictions = predictions

    def __str__(self):
        return f"Line: {self.name}, Color: {self.color}, Code:{self.code} Trains: {self.trains},  Stations: {len(self.stations)}, Trips: {self.trips}"

In [ ]:
trains = feed_rt["entity"]
trips = pd.read_csv('trips.txt')
stops = pd.read_csv('stops.txt')
stop_times = pd.read_csv('stop_times.txt')
prediction = predictions["Trains"]
prediction = pd.json_normalize(prediction)
prediction = prediction[prediction["Line"].isin(["OR", "SV", "BL"])].drop_duplicates()

#Initialise line values
def lines(name, code):
    color = name.upper()
    linetrains = []
    for train in trains:
        if train["vehicle"]["trip"]["routeId"] == color:
            linetrains.append(train)
    linetrains = pd.json_normalize(linetrains)
    linetrains["vehicle.stopId"] = linetrains["vehicle.stopId"].str[:-2]

    trip = trips[trips["route_id"]==color]
    times = stop_times[stop_times["trip_id"].isin(trip["trip_id"])]
    stations = stops[stops["stop_id"].isin(times["stop_id"])]
    stations = stations.drop_duplicates(subset="parent_station", keep="first")
    stations["stop_name"] = stations["stop_name"].str.split(", ").str[0]
    stations["stop_id"] = stations["stop_id"].str[:-2]
    

    line = Line(name, code, color, linetrains, stations, trip)

    return line

orangeline = lines("Orange", "OR")
silverline = lines("Silver", "SV")
blueline = lines("Blue", "BL")

#Order station list
def order_stations(line, filename):
    df1 = line.stations
    df2 = pd.read_csv(filename)
    df2["order"] = df2.index
    df2 = df2.rename(columns={"Stations":"stop_name"})
    df2["stop_name"] = df2["stop_name"].str.upper()
    stations = pd.merge(df1, df2, on="stop_name").sort_values('order').reset_index().drop(columns=["index", "order"])
   
    return stations

silverline.stations = order_stations(silverline, 'ordered_silver_list.txt')
orangeline.stations = order_stations(orangeline, 'ordered_orange_list.txt')
blueline.stations = order_stations(blueline, 'ordered_blue_list.txt')

#Map station values to the next stop ID in trains
def next_stop(line):
    stations = line.stations

    # Create next stop columns using a single shift operation for both IDs and names
    stations[['next_stop_id_0', 'next_stop_id_1']] = stations[['stop_id', 'stop_name']].shift(-1).combine_first(stations[['stop_id', 'stop_name']].shift(1))

    # Rename columns after shifting to make it clear
    stations["next_stop_name_0"] = stations["stop_name"].shift(-1)
    stations["next_stop_name_1"] = stations["stop_name"].shift(1)

    # Merge trains with stations, focusing on relevant columns to minimize memory usage
    trains = pd.merge(
        line.trains,
        stations[['stop_id', 'stop_name', 'next_stop_name_0', 'next_stop_name_1']],
        how='left',
        left_on='vehicle.stopId',
        right_on='stop_id'
    )
    
    return trains

orangeline.trains=next_stop(orangeline)
blueline.trains=next_stop(blueline)
silverline.trains=next_stop(silverline)

def minutes(line, prediction):
    stations = line.stations.copy()  # Avoid modifying the original DataFrame
    
    # Filter and process prediction data
    pred = (prediction[prediction["Line"] == line.code]
             .groupby(['Group', 'LocationCode', 'LocationName'])['Min']
             .apply(list)
             .reset_index())
    
    # Adjust Group values and rename columns for merging
    pred['Group'] = pred['Group'].astype(int) - 1
    pred = pred.pivot(index='LocationCode', columns='Group', values='Min').reset_index()
    pred.columns = ['LocationCode'] + [f'minute{group}' for group in pred.columns[1:]]

    # Extract LocationCode from stop_id
    stations['LocationCode'] = stations['stop_id'].str[3:]

    # Merge prediction results with stations
    stations = stations.merge(pred, on='LocationCode', how='left')

    return stations

orangeline.stations = minutes(orangeline, prediction)
silverline.stations = minutes(silverline, prediction)
blueline.stations = minutes(blueline, prediction)

In [9]:
import plotly.express as px
import plotly.graph_objects as go

px.set_mapbox_access_token("pk.eyJ1IjoiYWJoYXlwYWkiLCJhIjoiY20zOWV3Y2YwMTFlNDJqcHQ0ZmRxemI2ZyJ9.pwqjd5DXqnENZiejOwLR_Q")

In [10]:
def initialize_map(line):
        fig = px.scatter_mapbox(line.stations, lat = "stop_lat", lon="stop_lon")

        # Set layout for Mapbox
        fig.update_layout(
            autosize=True,
            hovermode='closest',
            map=dict(
                bearing=0,
                center=dict(
                    lat=38.92,
                    lon=-77.07
                ),
                pitch=0,
                zoom=10,
                style='light'
            ),
            showlegend=False,
            margin={"r":0,"t":0,"l":0,"b":0}  # Remove default margins
        )
        return fig


In [11]:
def plot(line, direction):
    fig = initialize_map(line)
    
    def plot_stations(line, direction):
        fig.add_trace(go.Scattermap(
            lat=line.stations.stop_lat,
            lon=line.stations.stop_lon,
            mode='markers+text+lines',
            marker=go.scattermap.Marker(size=9, color=line.name.lower()),  # Specify a color explicitly
            text=line.stations.stop_name,
            textfont=dict(size=8, color="black", family="Open Sans Bold")
        ))

        if direction:
            cur = line.stations.iloc[0]
            fig.add_trace(go.Scattermap(
                lat=[cur.stop_lat],
                lon=[cur.stop_lon],
                mode='markers+text',
                text="Destination: ",
                textfont=dict(size=11, color="black", family="Open Sans Bold"),
                marker=dict(size=20, symbol= "circle-stroked", color="red")
            ))
        else:
            cur = line.stations.iloc[-1]
            fig.add_trace(go.Scattermap(
                lat=[cur.stop_lat],
                lon=[cur.stop_lon],
                mode='markers+text',
                text="Destination",
                textfont=dict(size=11, color="black", family="Open Sans Bold"),
                marker=dict(size=20, symbol= "circle-stroked", color="red")
            ))


    def plot_trains(line, direction):
        trains = line.trains
        trains = trains[trains["vehicle.trip.directionId"]==direction]
        stopped_trains = trains[trains["vehicle.currentStatus"]=="STOPPED_AT"]
        trains = trains[trains["vehicle.currentStatus"]!="STOPPED_AT"]

        #For stopped trains
        fig.add_trace(go.Scattermap(
                lat=stopped_trains["vehicle.position.latitude"],
                lon=stopped_trains["vehicle.position.longitude"],
                mode='markers',
                text=stopped_trains["vehicle.currentStatus"]+" "+stopped_trains["stop_name"],
                marker=dict(size= 16, symbol= "rail-metro")
            ))
        
        #For trains in transit
        lat = trains["vehicle.position.latitude"]
        lon = trains["vehicle.position.longitude"]
        if direction:
            fig.add_trace(go.Scattermap(
                lat=lat,
                lon=lon,
                mode='markers',
                text=trains["vehicle.currentStatus"]+" "+trains["next_stop_name_1"],
                marker=dict(size= 16, symbol= "rail-metro")
            ))
        else:
            fig.add_trace(go.Scattermap(
                lat=lat,
                lon=lon,
                mode='markers',
                text=trains["vehicle.currentStatus"]+" "+trains["next_stop_name_0"],
                hoverinfo=['skip'],
                marker=dict(size= 16, symbol= "rail-metro")
            ))


    plot_stations(line, direction)
    plot_trains(line, direction)
    fig.show()


In [12]:
plot(orangeline, 0)

In [17]:
blueline.stations

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,location_type,parent_station,wheelchair_boarding,level_id,next_stop_id_0,next_stop_id_1,next_stop_name_0,next_stop_name_1,LocationCode
0,PF_J03,FRANCONIA-SPRINGFIELD,Blue Line to DOWNTOWN LARGO,38.766498,-77.167989,1,0,STN_J03,1,J03_L1,PF_J02,VAN DORN STREET,VAN DORN STREET,None,J03
1,PF_J02,VAN DORN STREET,Blue Line to Downtown Largo | to Franconia-Spr...,38.799285,-77.129266,71,0,STN_J02,1,J02_L1,PF_C13,KING ST-OLD TOWN,KING ST-OLD TOWN,FRANCONIA-SPRINGFIELD,J02
2,PF_C13,KING ST-OLD TOWN,Blue/Yellow Line to Downtown Largo/Fort Totten...,38.806597,-77.060968,1,0,STN_C13,1,C13_L1,PF_C12,BRADDOCK ROAD,BRADDOCK ROAD,VAN DORN STREET,C13
3,PF_C12,BRADDOCK ROAD,Blue/Yellow Line to Downtown Largo/Fort Totten...,38.814079,-77.053719,7,0,STN_C12,1,C12_L1,PF_C11,POTOMAC YARD/VT,POTOMAC YARD/VT,KING ST-OLD TOWN,C12
4,PF_C11,POTOMAC YARD/VT,Blue/Yellow Line to Franconia-Springfield/Hunt...,38.833199,-77.046494,1,0,STN_C11,1,C11_L1,PF_C10,RONALD REAGAN WASHINGTON NATIONAL AIRPORT,RONALD REAGAN WASHINGTON NATIONAL AIRPORT,BRADDOCK ROAD,C11
5,PF_C10,RONALD REAGAN WASHINGTON NATIONAL AIRPORT,Blue/Yellow Line to Franconia-Springfield/Hunt...,38.853485,-77.044094,11,0,STN_C10,1,C10_L1,PF_C09,CRYSTAL CITY,CRYSTAL CITY,POTOMAC YARD/VT,C10
6,PF_C09,CRYSTAL CITY,Blue/Yellow Line to Downtown Largo/Fort Totten,38.857786,-77.050559,11,0,STN_C09,1,C09_L2,PF_C08,PENTAGON CITY,PENTAGON CITY,RONALD REAGAN WASHINGTON NATIONAL AIRPORT,C09
7,PF_C08,PENTAGON CITY,Blue/Yellow Line to Franconia-Springfield/Hunt...,38.861852,-77.059582,43,0,STN_C08,1,C08_L2,PF_C07,PENTAGON,PENTAGON,CRYSTAL CITY,C08
8,PF_C07,PENTAGON,Blue/Yellow Line to Franconia-Springfield/Hunt...,38.869198,-77.054128,42,0,STN_C07,1,C07_L2,PF_C06,ARLINGTON CEMETERY,ARLINGTON CEMETERY,PENTAGON CITY,C07
9,PF_C06,ARLINGTON CEMETERY,Blue Line to Franconia-Springfield,38.884558,-77.063163,3,0,STN_C06,1,C06_L2,PF_C05,ROSSLYN,ROSSLYN,PENTAGON,C06
